In [41]:
pip install mediapipe

In [42]:
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp

In [43]:
# Load your trained ANN model (expects input shape (None, 63))
model = tf.keras.models.load_model("asl_ann_model1.keras")
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [44]:
# Mapping of class indices to labels (must match your training labels)
class_indices = {
    'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7,
    'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14,
    'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21,
    'W': 22, 'X': 23, 'Y': 24, 'Z': 25, 'del': 26, 'nothing': 27, 'space': 28
}
labels = {v: k for k, v in class_indices.items()}

In [45]:
# Initialize MediaPipe Hands for static image processing
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=True,    # For processing single images
    max_num_hands=1,
    min_detection_confidence=0.5
)

In [46]:
def predict_image(image_path):
    """
    Processes an image at the given path, extracts and normalizes hand landmarks,
    predicts the sign using the ANN model, and returns the predicted label along with the annotated image.
    """
    # Load image from disk
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Could not load image at path: " + image_path)

    # Make a copy for display/annotation
    display_image = image.copy()

    # Convert BGR to RGB (MediaPipe expects RGB)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    prediction_text = "No hand detected"
    if results.multi_hand_landmarks:
        # Process first detected hand
        hand_landmarks = results.multi_hand_landmarks[0]
        # Extract landmarks into an array of shape (21, 3)
        landmarks = np.array([[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark])
        # Normalize landmarks to obtain a 63-dimensional feature vector
        features = normalize_landmarks(landmarks)
        features = np.expand_dims(features, axis=0)  # shape: (1, 63)

        # Predict the sign using the ANN model
        predictions = model.predict(features)
        predicted_class = np.argmax(predictions[0])
        prediction_text = labels.get(predicted_class, "Unknown")

        # Draw hand landmarks on the display image for visual feedback
        h, w, _ = image.shape
        for lm in hand_landmarks.landmark:
            cx, cy = int(lm.x * w), int(lm.y * h)
            cv2.circle(display_image, (cx, cy), 3, (0, 255, 0), -1)

    # Overlay the predicted label on the display image
    cv2.putText(display_image, prediction_text, (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3)

    return prediction_text, display_image

In [65]:

# Example usage: Predict from a static image
image_path = "h_test.png"  # Replace with the path to your test image
predicted_label, annotated_image = predict_image(image_path)
print("Predicted Label:", predicted_label)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Predicted Label: G
